# Importing the data and saving it to .json

It's exported to json format because I wanted to mantain the real data, but it's better to use pickle format or parquet.

In [21]:
import pandas as pd
from numpy import NaN

In [22]:
source = "https://www.datos.gov.co/resource/vuyt-mqpw.json"
df = pd.read_json(source + "?$limit=600000")
df.to_json('../data/raw_data/raw_data.json')
df.to_csv('../data/raw_data/raw_data.csv')

In [23]:
df

,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,genero,grupo_etario,cantidad
0,ATLÁNTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
1,BOYACÁ,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
2,CAQUETÁ,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
3,CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
4,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
...,...,...,...,...,...,...,...,...
575716,VALLE,CARTAGO,76147000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,1
575717,VALLE,GUACARÍ,76318000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,1
575718,VALLE,GUADALAJARA DE BUGA,76111000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,1
575719,VALLE,PALMIRA,76520000,SIN EMPLEO DE ARMAS,28/02/2023,FEMENINO,ADULTOS,2


# Adjustments for columns, nulls and duplicates, (cleaning process).

## Null data treatment

Renaming columns to English language and dropping codigo_dane column

In [3]:
df = df.rename({'departamento':'department', 
           'municipio':'municipality', 
           'armas_medios':'gun_type', 
           'fecha_hecho':'date', 
           'genero':'sex', 
           'grupo_etario':'age_group', 
           'cantidad':'cases'}, axis = 1
).copy()

df = df.drop('codigo_dane', axis = 1).copy()

In [4]:
df.info(), df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575721 entries, 0 to 575720
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   department    575721 non-null  object
 1   municipality  575721 non-null  object
 2   gun_type      575721 non-null  object
 3   date          575721 non-null  object
 4   sex           575721 non-null  object
 5   age_group     574110 non-null  object
 6   cases         575721 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 30.7+ MB


(None,
 department      object
 municipality    object
 gun_type        object
 date            object
 sex             object
 age_group       object
 cases            int64
 dtype: object)

In [5]:
empty_info = df.isnull().sum()*100/df.shape[0]
print(f'{empty_info}\n\n{df.isnull().sum()}')

department      0.000000
municipality    0.000000
gun_type        0.000000
date            0.000000
sex             0.000000
age_group       0.279823
cases           0.000000
dtype: float64

department         0
municipality       0
gun_type           0
date               0
sex                0
age_group       1611
cases              0
dtype: int64


Eliminating null values from 'age_group' column

In [6]:
df = df[~df['age_group'].isnull()]

Erasing duplicates

In [7]:
df = df.loc[:, ~df.columns.duplicated()].copy()

## Data dtype changes

In [8]:
df['sex'] = df['sex'].astype('category')
df['age_group'] = df['age_group'].astype('category')
df['gun_type'] = df['gun_type'].astype('category')
df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y')

In [9]:
df.dtypes

department              object
municipality            object
gun_type              category
date            datetime64[ns]
sex                   category
age_group             category
cases                    int64
dtype: object

## Department column changes for future graphs

Changing codification of the column department to don't have errors

In [10]:
df.loc[:, 'department'] = df['department'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')

In [11]:
df['department'].replace({'SAN ANDRES':'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                             'VALLE':'VALLE DEL CAUCA',
                             'NARINO':'NARIÑO',
                             'GUAJIRA':'LA GUAJIRA'}, inplace = True)
df.department.unique()

array(['ATLANTICO', 'BOYACA', 'CAQUETA', 'CASANARE', 'CUNDINAMARCA',
       'SUCRE', 'VALLE DEL CAUCA', 'HUILA', 'ANTIOQUIA', 'ARAUCA',
       'BOLIVAR', 'CALDAS', 'CAUCA', 'CESAR', 'CHOCO', 'CORDOBA',
       'MAGDALENA', 'META', 'NARIÑO', 'NORTE DE SANTANDER', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'TOLIMA', 'VAUPES', 'GUAVIARE',
       'LA GUAJIRA', 'QUINDIO', 'AMAZONAS', 'VICHADA', 'GUAINIA',
       'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
       'NO REPORTA'], dtype=object)

Dropping data without department report

In [12]:
df['department'].replace('NO REPORTA', NaN, inplace = True)
df = df[~df['department'].isnull()]

Changing the name of Bogotá municipality and its department to plot maps later

In [13]:
df.loc[df['municipality'] == 'BOGOTÁ D.C. (CT)', 'department'] = 'SANTAFE DE BOGOTA D.C'

## Little columns changes

Artículo 365A del código penal colombiano

Parágrafo 1°. Para los efectos de esta ley, entiéndanse como arma blanca el objeto punzante, cortante, cortocontundente o cortopunzante apto para herir cortar, matar o dañar; que posea bordes filosos o punzantes, tales como navajas, puñales, puñaletas, punzones o cualquier objeto de similares características.

---

Article 365A of the Colombian Penal Code

Paragraph 1. For the purposes of this law, a white weapon is understood to be any sharp, cutting, blunt or sharp object suitable to wound, cut, kill or damage; that has sharp or pointed edges, such as knives, daggers, daggers, stabs, awls or any object of similar characteristics.

In [14]:
# Change on genre no reports
df['sex'] = df['sex'].replace('NO REPORTADO', 'NO REPORTA')

# Change on age_group no reports
df['age_group'] = df['age_group'].replace('NO REPORTADO', 'NO REPORTA')

# Change on gun_type based on Penal Code
df['gun_type'] = df['gun_type'].replace({'ARMA BLANCA / CORTOPUNZANTE':'ARMA BLANCA',
                            '-':'NO REPORTA',
                            'NO REPORTADO':'NO REPORTA',
                            'CORTOPUNZANTES':'ARMA BLANCA',
                            'CORTANTES':'ARMA BLANCA',
                            'CONTUNDENTES':'ARMA BLANCA',
                            'PUNZANTES':'ARMA BLANCA'})

In [15]:
print(df['gun_type'].unique(), 
      df['sex'].unique(), 
      df['age_group'].unique())

['ARMA BLANCA', 'ARMA DE FUEGO', 'NO REPORTA', 'SIN EMPLEO DE ARMAS', 'ESCOPOLAMINA']
Categories (5, object): ['ARMA BLANCA', 'ARMA DE FUEGO', 'ESCOPOLAMINA', 'NO REPORTA', 'SIN EMPLEO DE ARMAS'] ['MASCULINO', 'FEMENINO', 'NO REPORTA']
Categories (3, object): ['FEMENINO', 'MASCULINO', 'NO REPORTA'] ['ADULTOS', 'ADOLESCENTES', 'MENORES', 'NO REPORTA']
Categories (4, object): ['ADOLESCENTES', 'ADULTOS', 'MENORES', 'NO REPORTA']


## Dealing with outliers

(This change is bases on the distribution of variables analysis from Domestic-violenceEDA-v1 notebook)

As seen above and given that each row in the dataset represents a single record, I consider it appropriate to remove records where cases are more than 20, as it is absurd to think that a single incident of domestic violence has been reported with more than 20 victims of violence in one place. This could have been caused by a data entry error, as there are cases that exceed 100.

In [16]:
df[['cases']].query('cases > 20').count()

cases    3439
dtype: int64

In [17]:
df = df.query('cases < 20').reset_index(drop=True).copy()

# Exporting data as .parquet

In [18]:
df = df.reset_index(drop=True).copy()

In [19]:
df.to_parquet('../data/processed/data_cleaned.parquet', index=None)
df.to_csv('../data/processed/data_cleaned.csv', index=None)